# Retirement Simulation: Life Cycle Investing Strategies

## FINC 450: Life Cycle Investing

This notebook implements a Monte Carlo simulation comparing retirement spending strategies. Our goals are to understand:

1. How **duration matching** protects funded status against interest rate risk
2. How **variable consumption** reduces default probability
3. The trade-offs between these approaches

---

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import our simulation modules
from retirement_simulation import (
    EconomicParams,
    BondParams,
    SimulationParams,
    run_monte_carlo,
    compute_summary_stats,
    STRATEGIES,
    liability_pv,
    liability_duration,
    effective_duration,
    zero_coupon_price,
)
from visualizations import (
    plot_duration_matching_intuition,
    plot_strategy_comparison_bars,
    plot_wealth_paths_spaghetti,
    plot_consumption_paths,
    plot_final_wealth_distribution,
    plot_interest_rate_scenarios,
    create_summary_table,
)

%matplotlib inline
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 150

## 1. The Setup: A Retiree's Problem

Consider a retiree with:
- **Initial wealth**: $2.5 million
- **Annual spending need**: $100,000 (real)
- **Retirement horizon**: 30 years
- **Stock allocation**: 40% (fixed)

**Key question**: How should the retiree allocate the remaining 60% in bonds, and how should they adjust spending over time?

In [ ]:
# Simulation parameters
sim_params = SimulationParams(
    initial_wealth=2_500_000,
    annual_consumption=100_000,
    horizon=30,
    stock_weight=0.40,
    n_simulations=10_000,
    random_seed=42
)

# Economic environment parameters
econ_params = EconomicParams(
    r_bar=0.03,        # 3% long-run real rate
    phi=0.85,          # Interest rate persistence
    sigma_r=0.012,     # Rate volatility
    mu_excess=0.04,    # 4% equity risk premium
    sigma_s=0.18,      # Stock volatility (18%)
    rho=-0.2,          # Negative correlation: stocks fall when rates rise
)

# Bond parameters (maturities)
bond_params = BondParams(
    D_mm=0.25,         # Money market maturity (3-month zero-coupon)
    D_lb=15.0          # Long bond maturity (15-year zero-coupon)
)

print("Simulation Configuration:")
print(f"  Initial wealth: ${sim_params.initial_wealth:,.0f}")
print(f"  Annual consumption target: ${sim_params.annual_consumption:,.0f}")
print(f"  Horizon: {sim_params.horizon} years")
print(f"  Stock allocation: {sim_params.stock_weight*100:.0f}%")
print(f"  Number of simulations: {sim_params.n_simulations:,}")
print(f"\nMean reversion parameter: φ = {econ_params.phi}")

## 2. Understanding Liabilities

The retiree's **liability** is the present value of future consumption needs:

$$PV_{liab}(r) = C \times \frac{1 - (1+r)^{-T}}{r}$$

The **modified duration** of liabilities measures sensitivity to rate changes:

$$D_{liab}(r, T) = \frac{1}{PV_{liab}} \times \sum_{t=1}^{T} \frac{t \times C}{(1+r)^{t+1}}$$

In [ ]:
# Calculate liability PV and duration at different rates
rates = [0.01, 0.02, 0.03, 0.04, 0.05]
C = sim_params.annual_consumption
T = sim_params.horizon

print("Liability Analysis at Different Interest Rates:")
print("=" * 75)
print(f"{'Rate':>8} {'PV Liabilities':>18} {'Trad Duration':>15} {'Eff Duration':>15}")
print("-" * 75)

for r in rates:
    pv = liability_pv(C, r, T, r_bar=econ_params.r_bar, phi=econ_params.phi)
    trad_dur = liability_duration(C, r, T)  # Traditional (flat yield curve)
    eff_dur = liability_duration(C, r, T, r_bar=econ_params.r_bar, phi=econ_params.phi)
    print(f"{r*100:>7.1f}% {pv:>17,.0f} {trad_dur:>14.1f} yrs {eff_dur:>14.1f} yrs")

print("\nNote: Effective duration under mean reversion is much shorter than traditional duration.")
print("This is because long-term rates are anchored by mean reversion.")

In [ ]:
# Demonstrate effective duration under mean reversion
phi = econ_params.phi

print("Effective Duration of Zero-Coupon Bonds Under Mean Reversion")
print("=" * 65)
print(f"Mean reversion parameter: φ = {phi}")
print(f"Maximum possible duration: 1/(1-φ) = {1/(1-phi):.2f} years")
print()
print(f"{'Maturity (τ)':>15} {'Traditional Duration':>22} {'Effective Duration':>20}")
print("-" * 65)

for tau in [1, 5, 10, 15, 20, 30]:
    eff_dur = effective_duration(tau, phi)
    print(f"{tau:>12} yrs {tau:>22.1f} {eff_dur:>20.2f}")

print("\nKey insight: A 15-year zero has effective duration of only ~6 years!")
print("Duration matching requires this adjustment to be correct.")

## 2.5 Zero-Coupon Bond Pricing Under Mean Reversion

A critical insight: when interest rates **mean-revert**, long-term rates don't move 1-for-1 with short-term rates. This affects bond pricing and duration.

### The Key Formula

For the AR(1) process $r_{t+1} = \bar{r} + \phi(r_t - \bar{r}) + \varepsilon$, the **effective duration** of a τ-year zero-coupon bond is:

$$B(\tau) = \frac{1 - \phi^\tau}{1 - \phi}$$

This is the bond's sensitivity to changes in the **current short rate** (not parallel yield curve shifts).

### Why This Matters

With $\phi = 0.85$:
- A 1-year zero has effective duration = 1.0 (as expected)
- A 15-year zero has effective duration ≈ 6.1 (NOT 15!)
- Maximum possible duration = $\frac{1}{1-\phi}$ = 6.67 (regardless of maturity)

**Intuition**: Mean reversion "anchors" long-term rates near $\bar{r}$. A shock to today's rate has diminishing impact on rates far in the future.

## 3. The Four Strategies

We compare four retirement strategies that combine:

**Bond Allocation:**
- **Money Market (MM)**: All bonds in short-duration money market (D ≈ 0.25)
- **Duration Matching (DM)**: Match bond duration to liability duration

**Consumption Rule:**
- **Fixed**: Spend $100k/year regardless of wealth
- **Variable**: Spend a percentage of remaining wealth

| Strategy | Bond Duration | Consumption |
|----------|--------------|-------------|
| MM + Fixed | D ≈ 0 | Fixed $100k/yr |
| DurMatch + Fixed | D = D_liab | Fixed $100k/yr |
| MM + Variable | D ≈ 0 | % of wealth |
| DurMatch + Variable | D = D_liab | % of wealth |

In [ ]:
# Print strategy details
print("Strategies to Compare:")
print("=" * 50)
for i, strategy in enumerate(STRATEGIES, 1):
    print(f"{i}. {strategy}")

## 4. Run the Monte Carlo Simulation

We now simulate 10,000 retirement scenarios for each strategy.

In [ ]:
# Run the simulation
print("Running Monte Carlo simulation...")
results, rates, stock_returns = run_monte_carlo(
    econ_params=econ_params,
    bond_params=bond_params,
    sim_params=sim_params
)
print(f"Completed {sim_params.n_simulations:,} simulations for {len(STRATEGIES)} strategies.")

## 5. Results Summary

In [ ]:
# Display summary statistics
summary_df = compute_summary_stats(results)
print("\nStrategy Comparison Summary:")
print("=" * 80)
display(summary_df.round(2))

In [ ]:
# Visual comparison of key metrics
fig = plot_strategy_comparison_bars(results)
plt.suptitle('Strategy Performance Comparison', fontsize=14, y=1.02)
plt.show()

### Key Observations:

1. **Variable consumption eliminates default risk** - By definition, you can't run out of money if you spend a percentage of what you have.

2. **Fixed consumption has material default risk** - Even with $2.5M starting wealth, bad sequences of returns can lead to ruin.

3. **Duration matching has mixed effects** - It stabilizes funded status but introduces more portfolio volatility.

## 6. Understanding Duration Matching

Duration matching is an **immunization strategy** that aligns asset duration with liability duration.

**Why it works:**
- When rates fall: Bond prices rise, but so does the PV of liabilities
- When rates rise: Bond prices fall, but liability PV also falls

The **funded ratio** (Assets/Liabilities) stays stable.

In [ ]:
# Find an interesting scenario where rates move significantly
rate_changes = rates[:, -1] - rates[:, 0]  # Total rate change over 30 years

# Find a scenario with substantial rate increase
rising_rate_idx = np.argmax(rate_changes)
print(f"Sample path with rising rates:")
print(f"  Initial rate: {rates[rising_rate_idx, 0]*100:.2f}%")
print(f"  Final rate: {rates[rising_rate_idx, -1]*100:.2f}%")
print(f"  Rate change: {rate_changes[rising_rate_idx]*100:+.2f}%")

In [ ]:
# Duration matching intuition visualization
fig = plot_duration_matching_intuition(
    rates=rates,
    wealth_paths_mm=results['MM + Fixed'].wealth_paths,
    wealth_paths_dm=results['DurMatch + Fixed'].wealth_paths,
    consumption_target=sim_params.annual_consumption,
    horizon=sim_params.horizon,
    sample_idx=rising_rate_idx
)
plt.suptitle('Duration Matching Intuition: A Rising Rate Scenario', fontsize=14, y=1.02)
plt.show()

### Interpretation:

- **Top left**: Interest rate path over the 30-year horizon
- **Top right**: Money market strategy - assets and liabilities diverge when rates change
- **Bottom left**: Duration-matched strategy - assets track liabilities more closely
- **Bottom right**: Funded ratio comparison - duration matching provides more stable funding

**Caveat**: Duration matching introduces MORE portfolio volatility. It's a trade-off between funded-status stability and asset volatility.

## 7. Wealth Path Analysis

In [ ]:
# Spaghetti plot of wealth paths
fig = plot_wealth_paths_spaghetti(results, n_paths=100)
plt.suptitle('Wealth Trajectories by Strategy (100 Sample Paths)', fontsize=14, y=1.02)
plt.show()

### Observations:

- **Red paths** indicate scenarios that defaulted (hit zero)
- Variable consumption strategies never hit zero (by construction)
- Duration-matched strategies show wider dispersion in wealth outcomes
- The shaded region shows the 10th-90th percentile band

## 8. Consumption Path Analysis

In [ ]:
# Consumption paths
fig = plot_consumption_paths(results, sim_params.annual_consumption, n_paths=50)
plt.suptitle('Consumption Paths by Strategy', fontsize=14, y=1.02)
plt.show()

### Observations:

- **Fixed consumption** strategies maintain the target until default (then consumption drops to zero)
- **Variable consumption** strategies show more year-to-year volatility but never hit zero
- The trade-off: consumption certainty vs. default risk

## 9. Final Wealth Distribution

In [ ]:
# Final wealth distribution
fig = plot_final_wealth_distribution(results)
plt.show()

## 10. Interest Rate Scenarios

In [ ]:
# Visualize the interest rate scenarios
fig = plot_interest_rate_scenarios(rates, n_paths=100)
plt.show()

## 11. Key Teaching Points

### 1. Duration Matching Protects Funded Status

Duration matching ensures that when interest rates change, assets and liabilities move together. This protects the **funded status** (Assets/Liabilities ratio).

However, it introduces more **portfolio volatility** because long-duration bonds are more volatile than money market instruments.

### 2. Variable Consumption Eliminates Default Risk

By spending a percentage of wealth rather than a fixed amount, the retiree can never run out of money (mathematically impossible).

The cost is **consumption uncertainty** - spending varies year to year.

### 3. The Two Hedges Address Different Risks

| Strategy | Risk Hedged | Cost |
|----------|------------|------|
| Duration Matching | Interest rate risk | Higher portfolio volatility |
| Variable Consumption | Sequence-of-returns risk | Consumption uncertainty |

### 4. Low-Rate Environment Considerations

When rates are already low (near the floor):
- Rates are more likely to rise than fall further
- Duration matching may hurt because long bonds will lose value if rates rise
- The asymmetry of rate movements matters

## 12. Sensitivity Analysis: Starting Rate Environment

In [ ]:
# How do results change with different starting rates?
print("Running sensitivity analysis for different starting interest rates...")
print("(This may take a minute)")

starting_rates = [0.01, 0.03, 0.05]
sensitivity_results = {}

for r0 in starting_rates:
    results_r0, _, _ = run_monte_carlo(
        econ_params=econ_params,
        bond_params=bond_params,
        sim_params=SimulationParams(
            initial_wealth=sim_params.initial_wealth,
            annual_consumption=sim_params.annual_consumption,
            horizon=sim_params.horizon,
            stock_weight=sim_params.stock_weight,
            n_simulations=5000,  # Fewer sims for speed
            random_seed=42
        ),
        initial_rate=r0
    )
    sensitivity_results[r0] = results_r0
    print(f"  Completed r0 = {r0*100:.0f}%")

print("Done!")

In [ ]:
# Compare default rates across starting rate environments
print("\nDefault Rates by Starting Interest Rate:")
print("=" * 70)
print(f"{'Strategy':<25} {'r0=1%':>12} {'r0=3%':>12} {'r0=5%':>12}")
print("-" * 70)

for strategy_name in [str(s) for s in STRATEGIES]:
    rates_str = ""
    for r0 in starting_rates:
        default_rate = 100 * sensitivity_results[r0][strategy_name].defaulted.mean()
        rates_str += f"{default_rate:>12.1f}%"
    print(f"{strategy_name:<25}{rates_str}")

### Interpretation:

- **Lower starting rates** generally lead to higher default probabilities (lower expected returns)
- **Duration matching** may perform worse in low-rate environments if rates are likely to rise
- **Variable consumption** remains robust across all rate environments (no default by construction)

## 14. Random Walk Interest Rate Benchmark

As a benchmark comparison, we can model interest rates as a **random walk** instead of a mean-reverting process:

$$r_{t+1} = r_t + \text{drift} + \sigma_r \varepsilon_t$$

Under a random walk:
- Rates have no anchor (no long-run mean to revert to)
- Interest rate shocks persist forever
- Traditional duration = effective duration (no mean reversion adjustment)

This is useful for understanding how the mean reversion assumption affects our conclusions.

In [ ]:
# Import additional modules for random walk analysis
from retirement_simulation import (
    RandomWalkParams,
    run_median_path_simulation,
    run_random_walk_monte_carlo,
    simulate_interest_rates_random_walk,
)
from visualizations import (
    plot_wealth_allocation_lifecycle,
    plot_allocation_comparison_median_path,
    plot_random_walk_vs_mean_reverting,
    plot_interest_rate_models_comparison,
)

# Compare interest rate models
fig = plot_interest_rate_models_comparison(r0=0.03, n_periods=30, n_paths=100)
plt.show()

### Key Observations:

- **Mean-reverting rates** show narrowing uncertainty over time as rates are pulled toward the long-run mean (3%)
- **Random walk rates** show expanding uncertainty - shocks accumulate without a gravitational pull
- This has profound implications for duration and bond pricing

## 15. Wealth Allocation Lifecycle Under Median Returns

To understand how portfolio allocation evolves over a retiree's lifetime, we can examine the **median-path scenario** where median returns are realized each period. For symmetric distributions, the median shock is zero, giving us the expected trajectory.

This visualization shows:
1. How interest rates evolve
2. How portfolio allocation (stocks, money market, long bonds) changes
3. Wealth and consumption over the lifecycle
4. Funded ratio trajectory

In [ ]:
# Run median-path simulation for Duration Match + Variable strategy under random walk
median_result_rw = run_median_path_simulation(
    strategy=STRATEGIES[3],  # DurMatch + Variable
    r0=0.03,
    sim_params=sim_params,
    bond_params=bond_params,
    econ_params=econ_params,
    use_random_walk=True
)

# Visualize the wealth allocation lifecycle
fig = plot_wealth_allocation_lifecycle(median_result_rw)
plt.show()

print(f"\nMedian Path Summary ({median_result_rw.strategy_name}):")
print(f"  Initial wealth: ${median_result_rw.wealth[0]/1e6:.2f}M")
print(f"  Final wealth: ${median_result_rw.wealth[-1]/1e6:.2f}M")
print(f"  Total consumption: ${sum(median_result_rw.consumption)/1e6:.2f}M")

### Observations from the Lifecycle Visualization:

- **Top-left**: Under a random walk, interest rates stay constant in the median scenario (no mean reversion pull)
- **Top-right**: Portfolio allocation shows the stock weight is fixed at 40%, while the bond allocation is split between money market and long bonds based on duration matching
- **Bottom-left**: Wealth grows over time as returns exceed consumption, then gradually declines as the retiree ages
- **Bottom-right**: The funded ratio starts above 1 (overfunded) and evolves as the liability shortens

## 16. Comparing All Strategies Under Random Walk

Let's compare how all four strategies evolve under the random walk interest rate model:

In [ ]:
# Compare all strategies under random walk model
fig = plot_allocation_comparison_median_path(
    strategies=STRATEGIES,
    r0=0.03,
    sim_params=sim_params,
    bond_params=bond_params,
    econ_params=econ_params,
    use_random_walk=True
)
plt.show()

## 17. Random Walk vs Mean-Reverting: Side-by-Side Comparison

Finally, let's directly compare how the same strategy performs under both interest rate models:

In [ ]:
# Compare random walk vs mean-reverting for the Duration Match + Variable strategy
fig = plot_random_walk_vs_mean_reverting(
    r0=0.03,
    sim_params=sim_params,
    bond_params=bond_params,
    econ_params=econ_params,
    strategy=STRATEGIES[3]  # DurMatch + Variable
)
plt.show()

print("\nKey difference: Under mean reversion, rates converge to the long-run mean (3%)")
print("while under random walk, rates remain at the initial level in the median scenario.")

## 18. Human Capital and Lifecycle Asset Allocation

The analysis above focused on a **retiree** with fixed stock allocation. But how should investors allocate throughout their **entire working life**?

### The Key Insight: Human Capital as an Asset

**Human Capital** = Present Value of Future Net Labor Income

A young worker's total wealth is mostly **Human Capital**:
- At age 25: Financial Wealth might be $10k, but Human Capital could be $3M+
- At age 65: Human Capital = $0 (retired), all wealth is financial

### The Merton Framework

The optimal fraction of **total wealth** in stocks (Merton Share):

$$w^* = \frac{\mu - r_f}{\gamma \sigma^2}$$

where:
- $\mu - r_f$ = equity risk premium
- $\gamma$ = risk aversion coefficient  
- $\sigma$ = stock volatility

### Asset Location Problem

If Human Capital has stock-like characteristics ($\beta_{labor} > 0$), it provides implicit equity exposure:

$$\text{Total Stock} = \text{Financial Stock} + \beta_{labor} \times \text{Human Capital}$$

Solving for optimal **financial portfolio** stock allocation:

$$w_{fin} = \frac{w^* \times TW - \beta_{labor} \times HC}{FW}$$

### The Lifecycle Pattern

This creates a characteristic **hump-shaped** or **flat-then-falling** equity glide path:
- **Young workers**: High HC → financial portfolio can be 100% stocks
- **Mid-career**: HC declining → equity allocation gradually falls
- **Retirement**: HC = 0 → financial portfolio equals Merton share

In [ ]:
# Import lifecycle simulation components
from retirement_simulation import (
    LifecycleParams,
    calculate_human_capital,
    get_merton_share,
    solve_financial_stock_allocation,
    run_lifecycle_simulation,
)
from visualizations import (
    plot_lifecycle_allocation,
    plot_lifecycle_comparison_beta,
    plot_lifecycle_sensitivity_gamma,
)

# Define lifecycle parameters
lifecycle_params = LifecycleParams(
    current_age=25,
    retirement_age=65,
    life_expectancy=90,
    current_wage=250_000,       # $250k annual wage
    wage_growth=0.00,           # Real wage growth
    subsistence=100_000,        # $100k subsistence consumption
    beta_labor=0.0,             # Bond-like human capital (professor, government)
    gamma=2.0,                  # Risk aversion
    initial_wealth=10_000       # Starting financial wealth
)

# Calculate Merton share for reference
merton_share = get_merton_share(
    mkt_excess=econ_params.mu_excess,
    sigma=econ_params.sigma_s,
    gamma=lifecycle_params.gamma
)
print(f"Merton optimal stock share (total wealth): {merton_share*100:.1f}%")
print(f"\nLifecycle Parameters:")
print(f"  Working ages: {lifecycle_params.current_age} to {lifecycle_params.retirement_age}")
print(f"  Net labor income: ${lifecycle_params.current_wage - lifecycle_params.subsistence:,}/year")
print(f"  Risk aversion (γ): {lifecycle_params.gamma}")

In [ ]:
# Run the lifecycle simulation (deterministic median path)
result = run_lifecycle_simulation(
    lifecycle_params=lifecycle_params,
    econ_params=econ_params
)

# Visualize the complete lifecycle allocation
fig = plot_lifecycle_allocation(result, lifecycle_params)
plt.show()

print(f"\nLifecycle Summary (β_labor = {lifecycle_params.beta_labor}):")
print(f"  Age 25 - Stock allocation: {result.stock_allocation[0]*100:.0f}%")
print(f"  Age 45 - Stock allocation: {result.stock_allocation[20]*100:.1f}%")
print(f"  Age 65 - Stock allocation: {result.stock_allocation[40]*100:.1f}%")
print(f"  Age 80 - Stock allocation: {result.stock_allocation[55]*100:.1f}%")

## 19. Effect of Labor Income Beta

The **beta of labor income** ($\beta_{labor}$) captures how correlated your human capital is with the stock market:

| $\beta_{labor}$ | Occupation Type | Human Capital Acts Like |
|----------------|-----------------|------------------------|
| 0.0 | Tenured professor, government | **Bonds** |
| 0.5 | Mid-level corporate | **Mixed** |
| 1.0 | Commissioned salesperson, startup | **Stocks** |

**Key insight**: If your human capital is already stock-like ($\beta_{labor} = 1$), you don't need as much equity in your financial portfolio!

In [ ]:
# Compare lifecycle allocation for different labor income betas
fig = plot_lifecycle_comparison_beta(
    econ_params=econ_params,
    betas=[0.0, 0.3, 0.6, 1.0],
    figsize=(14, 10)
)
plt.show()

## 20. Effect of Risk Aversion

The **risk aversion coefficient** ($\gamma$) determines the Merton share:

$$w^* = \frac{\mu - r_f}{\gamma \sigma^2}$$

Higher risk aversion → Lower equity allocation throughout life.

In [ ]:
# Compare lifecycle allocation for different risk aversion levels
fig = plot_lifecycle_sensitivity_gamma(
    econ_params=econ_params,
    gammas=[1.0, 2.0, 4.0, 8.0],
    figsize=(14, 10)
)
plt.show()

# Show Merton shares for each gamma
print("\nMerton Optimal Shares by Risk Aversion:")
print("-" * 45)
for g in [1.0, 2.0, 4.0, 8.0]:
    ms = get_merton_share(econ_params.mu_excess, econ_params.sigma_s, g)
    print(f"  γ = {g:.1f}: Merton share = {ms*100:.1f}%")

## 21. Connecting Lifecycle to Retirement

The **lifecycle model** (Sections 18-20) and the **retirement model** (Sections 1-17) are complementary:

| Phase | Model | Key Question |
|-------|-------|--------------|
| **Working Years** (25-65) | Lifecycle with Human Capital | How much equity given my human capital? |
| **Retirement** (65+) | Retirement Simulation | How to allocate bonds and manage consumption? |

### Key Takeaways

1. **Human Capital dominates early**: Young investors should hold high equity (100%) because their bond-like human capital provides implicit fixed income.

2. **Glide path emerges naturally**: The "target-date fund" pattern isn't arbitrary—it follows from optimal total wealth allocation as HC depletes.

3. **Occupation matters**: A tenured professor ($\beta=0$) should hold MORE equity than a commissioned salesperson ($\beta=1$).

4. **At retirement, HC=0**: The retirement analysis (fixed 40% equity) represents one point on the lifecycle continuum—the phase when human capital is exhausted.

5. **Risk aversion personalizes**: The Merton share is your "anchor" equity allocation; lifecycle effects cause deviations early in life.

## 22. Summary and Discussion Questions

### Summary Table

---

## Appendix: Model Details

### Economic Environment (VAR Structure)

**Interest Rates** follow an AR(1) process:
$$r_{t+1} = \bar{r} + \phi(r_t - \bar{r}) + \varepsilon_{r,t}$$

**Stock Returns** equal the risk-free rate plus an IID excess return:
$$R_{stock,t} = r_t + \mu_{excess} + \varepsilon_{s,t}$$

**Correlation**: Rate shocks and stock shocks have correlation $\rho = -0.2$

### Zero-Coupon Bond Pricing (Consistent with Mean Reversion)

Under the mean-reverting rate process, the price of a $\tau$-year zero-coupon bond is:

$$P(\tau) = \exp\left(-\tau \bar{r} - B(\tau)(r - \bar{r})\right)$$

where the **effective duration** is:
$$B(\tau) = \frac{1 - \phi^\tau}{1 - \phi}$$

The one-period return on holding a $\tau$-year zero is:
$$R_{bond,t} = \frac{P_{t+1}(\tau-1)}{P_t(\tau)} - 1$$

### Liability Valuation

Liabilities are valued using the same term structure:
$$PV_{liab} = \sum_{t=1}^{T} C \times P(t)$$

The **effective duration of liabilities** (sensitivity to short rate):
$$D_{liab}^{eff} = \frac{1}{PV_{liab}} \sum_{t=1}^{T} C \times P(t) \times B(t)$$

### Parameters Used

| Parameter | Value | Description |
|-----------|-------|-------------|
| $\bar{r}$ | 3% | Long-run mean real rate |
| $\phi$ | 0.85 | Rate persistence |
| $\sigma_r$ | 1.2% | Rate shock volatility |
| $\mu_{excess}$ | 4% | Equity risk premium |
| $\sigma_s$ | 18% | Stock return volatility |
| $\rho$ | -0.2 | Shock correlation |
| $\tau_{mm}$ | 0.25 | Money market maturity |
| $\tau_{lb}$ | 15 | Long bond maturity |
| $B(\tau_{lb})$ | 6.08 | Long bond effective duration |

### Discussion Questions for Class

1. **Risk trade-offs**: A retiree can choose between default risk (fixed consumption) and consumption uncertainty (variable consumption). Which would you prefer? Does your answer depend on other sources of income (Social Security, pensions)?

2. **Duration matching in practice**: Given that liability duration decreases as the retiree ages, how frequently should the portfolio be rebalanced? What are the transaction costs?

3. **Low-rate environments**: How should the analysis change if we believe rates are more likely to rise than fall from current levels? Does duration matching become less attractive?

4. **Bequest motives**: How would the optimal strategy change if the retiree cares about leaving wealth to heirs?

5. **Human capital and flexibility**: Young retirees might have the option to return to work if wealth falls. How does this "human capital option" affect optimal strategy choice?

6. **Beyond the model**: What important real-world considerations are missing from this simulation? (Hint: inflation uncertainty, health shocks, housing, Social Security timing, etc.)